In [2]:
import os
import glob
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import random

In [5]:
annotation_dir = r"C:\Users\cuongnt\Downloads\data\Annotation"
image_dir = r"C:\Users\cuongnt\Downloads\data\Images"

In [6]:
def get_image_path(annotation_path):
    path_items = annotation_path.split(os.path.sep)
    return os.path.join(image_dir, path_items[-2], path_items[-1] + '.jpg')

# Create CSV file

In [8]:
images_dir = r"C:\Users\cuongnt\Downloads\ubuntu\ssd_keras\data"

In [17]:
def get_labels(annotation_path):
    list_bndboxes = []
    xml = ET.parse(annotation_path).getroot()
    filename = os.path.basename(get_image_path(annotation_path))
    for obj in xml.findall("object"):
        bndbox = obj.find("bndbox")
        xmin, ymin, xmax, ymax = bndbox.find("xmin"), bndbox.find("ymin"), bndbox.find("xmax"), bndbox.find("ymax")
        xmin, ymin, xmax, ymax = int(xmin.text), int(ymin.text), int(xmax.text), int(ymax.text)
        list_bndboxes.append([filename, xmin, xmax, ymin, ymax, 1])
    return list_bndboxes

In [10]:
all_annotation_files = glob.glob(r'C:\Users\cuongnt\Downloads\data\Annotation\**\*', recursive = True)
all_annotation_files = [f for f in all_annotation_files if os.path.isfile(f)]

In [11]:
dataset = random.sample(all_annotation_files, 3000)

In [12]:
#Ensure all image are in jpeg format
for ant in dataset:
    img_path = get_image_path(ant)
    img = cv2.imread(img_path)
    cv2.imwrite(os.path.join(images_dir, os.path.basename(img_path)), img)

In [13]:
train_ds = dataset[: 2000]
eval_ds = dataset[2000: 2500]
test_ds = dataset[2500: ]

In [14]:
import csv

In [15]:
label_files = ['labels_train.csv', 'labels_val.csv', 'labels_test.csv']
ds = [train_ds, eval_ds, test_ds]

In [18]:
for (label_file, d) in zip(label_files, ds):
    labels = []
    for ant in d:
        boxes = get_labels(ant)
        labels.extend(boxes)
    with open(label_file, 'w', newline='') as csvFile:
        writer = csv.writer(csvFile, delimiter=',')
        writer.writerow(['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'])
        writer.writerows(labels)